In [ ]:
import pywt
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def text_to_bits(text: str) -> str:
    return ''.join(f"{ord(c):08b}" for c in text)

def bits_to_text(bits: str) -> str:
    # обрезаем до целых байт
    bits = bits[: (len(bits) // 8) * 8]
    chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
    return ''.join(chr(int(b, 2)) for b in chars)

def int_to_bits(n: int, length: int) -> str:
    return format(n, '0{}b'.format(length))

def bits_to_int(bits: str) -> int:
    return int(bits, 2)

In [ ]:
def embed_text(image_path: str, out_path: str, text: str, Q: int = 4):
    # ORIGINAL
    # img = Image.open(image_path).convert("L")
    # arr = np.array(img, dtype=np.float32)
    #

    # test RGB
    img = Image.open(image_path).convert('RGB')
    img_array = np.array(img)
    img_ycbcr = cv2.cvtColor(img_array, cv2.COLOR_RGB2YCrCb)
    arr = img_ycbcr[:, :, 0].astype(np.float32)
    #

    # DWT (используем periodization для более стабильного поведения)
    coeffs2 = pywt.dwt2(arr, 'haar', mode='periodization')
    LL, (LH, HL, HH) = coeffs2

    # flatten HL для простой индексации
    hl = HL.flatten()

    bits_payload = text_to_bits(text)
    payload_len_bytes = len(text)
    # 32-битный заголовок длины (в байтах)
    header_bits = int_to_bits(payload_len_bytes, 32)
    bits = header_bits + bits_payload

    print(f"Embedded text: {len(bits)}/{hl.size}")
    if len(bits) > hl.size:
        raise ValueError(f"Не хватает места в HL: нужно {len(bits)} бит, а доступно {hl.size} бит.")

    # Квантование: используем округление, чтобы симметрично работать с отриц. значениями
    for idx, bit in enumerate(bits):
        c = float(hl[idx])
        q = int(np.round(c / Q))   # <-- important: round, not floor
        # выставляем четность q в соответствии с битом
        if bit == '0':
            if q % 2 == 1:
                q -= 1
        else:
            if q % 2 == 0:
                q += 1
        hl[idx] = q * Q

    # записываем обратно и делаем IDWT
    HL_emb = hl.reshape(HL.shape)
    coeffs2_emb = (LL, (LH, HL_emb, HH))
    arr2 = pywt.idwt2(coeffs2_emb, 'haar', mode='periodization')

    arr2 = np.clip(arr2, 0, 255).astype(np.uint8)
    # test RGB
    img_ycbcr[:, :, 0] = arr2
    arr2 = cv2.cvtColor(img_ycbcr, cv2.COLOR_YCrCb2RGB)
    #

    Image.fromarray(arr2).save(out_path)
    print(f"Встраивание завершено. Сохранено: {out_path}")

In [ ]:
def extract_text(image_path: str, Q: int = 4) -> str:
    # ORIGINAL
    # img = Image.open(image_path).convert("L")
    # arr = np.array(img, dtype=np.float32)
    #

    # test RGB
    img = Image.open(image_path).convert('RGB')
    img_array = np.array(img)
    img_ycbcr = cv2.cvtColor(img_array, cv2.COLOR_RGB2YCrCb)
    arr = img_ycbcr[:, :, 0].astype(np.float32)
    #

    LL, (LH, HL, HH) = pywt.dwt2(arr, 'haar', mode='periodization')

    hl = HL.flatten()

    # сначала читаем 32 бита заголовка
    if hl.size < 32:
        raise ValueError("HL слишком маленький для чтения заголовка длины.")

    header_bits = ""
    for i in range(32):
        c = float(hl[i])
        q = int(np.round(c / Q))
        header_bits += '1' if (q % 2 == 1) else '0'

    payload_len_bytes = bits_to_int(header_bits)
    bits_needed = payload_len_bytes * 8

    if 32 + bits_needed > hl.size:
        raise ValueError(f"Невозможно прочитать весь payload: требуется {32 + bits_needed} коэффициентов, доступно {hl.size}")

    payload_bits = ""
    for i in range(32, 32 + bits_needed):
        c = float(hl[i])
        q = int(np.round(c / Q))
        payload_bits += '1' if (q % 2 == 1) else '0'

    return bits_to_text(payload_bits)

In [ ]:
used_image = 'img/marsik-960x1280.png'
output_image = 'embedded/marsik-960x1280-embedded.png'

# used_image = 'img/marsik-600x800.png'
# output_image = 'embedded/marsik-600x800-embedded.png'

# used_image = 'img/marsik-300x400.png'
# output_image = 'embedded/marsik-300x400-embedded.png'

# used_image = 'img/marsik-150x200.png'
# output_image = 'embedded/marsik-150x200-embedded.png'

# used_image = 'img/cat.jpg'
# output_image = 'embedded/cat-embedded.jpg'

# used_image = 'img/forest.jpg'
# output_image = 'embedded/forest-embedded.jpg'

secret_string = "Silence in an era of noise: How to find yourself in a world that never stops Our world is immersed in a continuous, intrusive hum. This is not just the physical noise of megacities, but a fundamental information and social backdrop that has become our new habitat. We wake up to the vibration of our smartphones, scroll through our news feeds at breakfast, and are immersed in a whirlwind of notifications, messaging apps, and endless online meetings at work. In the evening, as we try to relax, we unconsciously scroll through social media, consuming the carefully curated lives of others. This permanent digital noise creates the illusion of hyper-connection, but paradoxically leads to deep loneliness and distraction. We know more about the world, but less about ourselves. We have thousands of \"friends,\" but sometimes we don't have anyone to share our true sadness or joy with."

In [ ]:
embed_text(used_image, output_image, secret_string, Q=10)
result = extract_text(output_image, Q=10)
print("Извлечённый текст:", result)

In [ ]:
def compare_strings(str1, str2):
    """Сравнивает две строки и показывает различия"""
    if len(str1) != len(str2):
        print(f"Длины разные: {len(str1)} vs {len(str2)}")
    
    min_len = min(len(str1), len(str2))
    differences = []
    
    for i in range(min_len):
        if str1[i] != str2[i]:
            differences.append((i, str1[i], str2[i]))
    
    if differences:
        print("Найдены различия:")
        for pos, char1, char2 in differences:
            print(f"  Позиция {pos}: '{char1}' vs '{char2}'")
    else:
        print("Строки идентичны (в пределах общей длины)")
    
    return len(differences) == 0 and len(str1) == len(str2)

In [ ]:
print(len(secret_string))
print(len(result))

compare_strings(secret_string, result)